In [ ]:
!pip install qiskit
from qiskit import *
from qiskit.visualization import plot_histogram

In [ ]:
from math import ceil, floor, log, asin, sqrt, pi

In [ ]:
def XNOR(qc,in1,in2,out):
  qc.cx(in1,out)
  qc.cx(in2,out)
  qc.x(out)

In [ ]:
def create_vertices_qubits(V, n):
  serial = '0'
  vertices_register_list = []
  for i in range(V):
    label = 'v' + serial
    q = QuantumRegister(n,name = label)
    vertices_register_list.append(q)
    serial = chr(ord(serial)+1)
  return vertices_register_list

In [ ]:
def create_edges_qubits(E,n):
  serial = '0'
  edges_register_list = []
  for i in E:
    label = 'e' + serial
    q = QuantumRegister(n,name = label)
    edges_register_list.append(q)
    serial = chr(ord(serial)+1)
  return edges_register_list

In [ ]:
def create_output_qubits(E):
  return QuantumRegister(len(E),"check")

In [ ]:
def create_adjacency_list(V,E):
  adjacency = [[]for i in range(V)]
  for edge in E:
    adjacency[edge[0]].append(edge[1])
    adjacency[edge[1]].append(edge[0])
  return adjacency

In [ ]:
def oracle(qc,vertices,edges,outputs,kickback,E,n):
  for ind, e in enumerate(E):
    v1 = vertices[e[0]]
    v2 = vertices[e[1]]
    out = edges[ind]
    for i in range(n):
      XNOR(qc,v1[i],v2[i],out[i])
    qc.mct(out,outputs[ind])
    qc.x(outputs[ind])
  qc.mct(outputs,kickback)
  for ind, e in enumerate(E):
    v1 = vertices[e[0]]
    v2 = vertices[e[1]]
    out = edges[ind]
    qc.x(outputs[ind])
    qc.mct(out,outputs[ind])
    for i in range(n):
      XNOR(qc,v1[i],v2[i],out[i])

In [ ]:
def diffuser(qc,vertices,V,n):
  for i in vertices:
    qc.h(i)
    qc.x(i)
  qc.h(vertices[V-1][n-1])
  qc.mct(list(range(V*n-1)),V*n-1)
  qc.h(vertices[V-1][n-1])
  for i in vertices:
    qc.x(i)
    qc.h(i)

In [ ]:
def chromatic_upper_bound(adjacency):
  max_degree = 0
  for l in adjacency:
    if len(l) > max_degree:
      max_degree = len(l)
  n = ceil(log(max_degree+1,2))
  return n

In [ ]:
def number_of_iterations(V,n):
  number_of_qubits = V*n
  number_of_solutions = 2**number_of_qubits
  theta = asin(1/sqrt(number_of_solutions))
  number_of_iterations = ceil((pi/(4*theta))-0.5)
  return number_of_iterations

In [ ]:
def graph_coloring(V,E):
  adjacency = create_adjacency_list(V,E)
  n = chromatic_upper_bound(adjacency)
  iterations = number_of_iterations(V,n)

  vertices = create_vertices_qubits(V,n)
  edges = create_edges_qubits(E,n)
  outputs = create_output_qubits(E)
  kickback = QuantumRegister(1,"k")
  measurements = ClassicalRegister(n*V,"m")

  qc = QuantumCircuit()
  for v in vertices:
    qc.add_register(v)
  for e in edges:
    qc.add_register(e)
  qc.add_register(outputs)
  qc.add_register(kickback)
  qc.add_register(measurements)

  qc.x(kickback)
  qc.h(kickback)
  for v in vertices:
    qc.h(v)
  qc.barrier()

  for i in range(iterations):
    oracle(qc,vertices,edges,outputs,kickback,E,n)
    qc.barrier()
    diffuser(qc,vertices,V,n)
  
  qc.measure(list(range(V*n)),measurements)
  print(iterations)
  return qc

In [ ]:
V = 2
E = [[0,1]]
circuit = graph_coloring(V,E)
circuit.draw()

In [ ]:
sim = Aer.get_backend('aer_simulator')
job = execute(circuit, backend=sim, shots=10000)
result = job.result()
counts = result.get_counts()
plot_histogram(counts)